# Clean HatEval

This is a script to clean the HatEval dataset and to separate it into the two different tasks that we will be using it for. I'm assuming that it is saved to ``data/hateval2019``.

In [ ]:
import pandas as pd

We establish the path to our data and the generic name that our preprocessed datasets have.

In [ ]:
path = "../data/hateval2019/"
files = "hateval2019_clean_es_"

The first task is hate speech detection, so we only keep the HS label from the dataset.

In [ ]:
for i in ["train","dev","test"]:
    df = pd.read_csv(path+files+i+".csv")
    
    task1 = df[["id","text","HS"]]
    task1.columns = ["id","text","label"]
    task1.to_csv(path+"task1_es_"+i+".csv")

The second task requires us to only consider hate Tweets. Here we have to determine whether they are targeted or not and if they are agressive or not. We decided to make this task a multilabel classification task instead of a multiclass one, so we set the following labels:
1. Neither targeted nor agressive
1. Agressive but not targeted
1. Targeted but not agressive
1. Targeted and agressive

In [ ]:
for i in ["train","dev","test"]:
    df = pd.read_csv(path+files+i+".csv")
    
    task2 = df[["id","text","TR","AG"]].loc[df["HS"]==1]
    task2["label"] = 0
    task2["label"].loc[(task2.TR == 0) & (task2.AG == 1)] = 1
    task2["label"].loc[(task2.TR == 1) & (task2.AG == 0)] = 2
    task2["label"].loc[(task2.TR == 1) & (task2.AG == 1)] = 3
    print(task2.label.unique())
    task2 = task2.drop(["TR","AG"],axis=1)
    task2.to_csv(path+"task2_es_"+i+".csv")